In [2]:
import pandas as pd
import os
import csv
import numpy as np

In [3]:
#dataframe of all respiratory deaths at the state level
state_data = pd.read_csv('Underlying Cause of Death, 1999-2017_allyears_states.txt', sep='\t')

In [4]:
#dataframe of all respiratory deaths at the state level
state_abrevs = pd.read_csv('state_abrevs.csv', sep=',')

In [5]:
#Dataframe of state populations from 2000-2010
state_pops_2000 = pd.read_csv('co-est00int-tot.csv', encoding='latin-1')
#Dataframe of state populations from 2010-2018
state_pops_2010 = pd.read_csv('co-est2018-alldata.csv', encoding='latin-1')

In [6]:
def changing_col_name(dataframe, name_before, name_after):
    '''This function changes the name of one column and returns the dataframe.
    
    The column names must be input as strings.
    '''
    
    return dataframe.rename(columns={name_before: name_after})

In [7]:
def split_column(dataframe, column1, column2, column3, splitter):
    '''This function creates a new dataframe that has one specified column from the original dataframe.
    Then it adds two new columns to the original dataframe (or replaces one of the original columns if the names are 
    the same) with the split values.
    
    The column name inputs must be strings.
    The splitter is the symbol or letter that is used to split the string.
    '''
    
    new_df = dataframe[column1].str.split(splitter, n=1, expand=True)
    dataframe[column2] = new_df[0]
    dataframe[column3] = new_df[1]
    
    return dataframe
    

In [8]:
def choose_data_by_year(dataframe, year):
    '''This function takes a dataframe as an input and the target year as an integer, and outputs a dataframe that contains 
    only that specific year.   
    
    It also specifies which columns we want to keep in the final dataframe, and it renames the population column
    so that it does not have a year-specific name.
    '''
    return dataframe[['County',
       'POPESTIMATE'+str(year),
       'County Code', 'Deaths', 'Month', 'Month Code',
       'State', 'Year']].loc[dataframe["Year"]==str(year)].rename(columns={'POPESTIMATE'+str(year):'Population'})
    
    
    


In [9]:
def concat_dfs_vertically(dataframe1, dataframe2):
    '''This function merges two dataframes vertically.
    It assumes they have the same column names already.
    '''
    return pd.concat([dataframe1, dataframe2])
    
    

In [10]:
#Importing the .csv files for death rates from each year and merging them into one df

counties_data = pd.DataFrame()
for file in os.listdir('county_data'):
    new_year = pd.read_csv('county_data/' + file, sep='\t', na_filter = False)
    counties_data = pd.concat([counties_data, new_year], axis=0)


/Users/Marta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [11]:
#Getting rid of extra columns in death rates df

col_list = ['Notes', 'Population', 'Crude Rate', '% of Total Deaths']
for name in col_list:
    counties_data = counties_data.drop(columns = name)

In [12]:
#Splitting up column values into two columns in death rates df
counties_data = split_column(counties_data, 'County', 'County', 'State Abr', ',')
counties_data = split_column(counties_data, 'Month Code', 'Year', 'Month', '/')

In [13]:
#Changing the column name to match in the health data frame
state_abrevs = changing_col_name(state_abrevs, 'State Abrev', 'State Abr')

In [14]:
#removing the extra space from before the string in the state abrevs column
counties_data['State Abr'] = counties_data['State Abr'].str.lstrip()

In [15]:
#Making both the state abbreviation columns utf-8 encoding
counties_data['State Abr'] = counties_data['State Abr'].str.encode('utf-8')
state_abrevs['State Abr'] = state_abrevs['State Abr'].str.encode('utf-8')

In [16]:
#Merging the county health data and the state abbreviations
county_data_merge = pd.merge(counties_data, state_abrevs, on=['State Abr'])

In [17]:
#Dropping extra State column that contains Nans
county_data_merge.drop(columns = ['State_x', 'State Code'])

,County,County Code,Deaths,Month,Month Code,State Abr,Year,State_y
0,Baldwin County,01003,13,01,2012/01,b'AL',2012,Alabama
1,Baldwin County,01003,11,02,2012/02,b'AL',2012,Alabama
2,Baldwin County,01003,18,03,2012/03,b'AL',2012,Alabama
3,Baldwin County,01003,16,05,2012/05,b'AL',2012,Alabama
4,Baldwin County,01003,11,06,2012/06,b'AL',2012,Alabama
5,Baldwin County,01003,11,07,2012/07,b'AL',2012,Alabama
6,Baldwin County,01003,13,08,2012/08,b'AL',2012,Alabama
7,Baldwin County,01003,10,09,2012/09,b'AL',2012,Alabama
8,Baldwin County,01003,11,10,2012/10,b'AL',2012,Alabama
9,Baldwin County,01003,16,11,2012/11,b'AL',2012,Alabama


In [18]:
county_data_merge = changing_col_name(county_data_merge, 'State_y', 'State')

In [19]:
#Changing column names to match between death rates and population dataframes
state_pops_2010 = changing_col_name(state_pops_2010, 'CTYNAME', 'County')
state_pops_2000 = changing_col_name(state_pops_2000, 'CTYNAME', 'County')
state_pops_2010 = changing_col_name(state_pops_2010, 'STNAME', 'State')
state_pops_2000 = changing_col_name(state_pops_2000, 'STNAME', 'State')

In [20]:
#Merging county population data for 2000 and 2010 with county death rates
county_pop_merge = pd.merge(state_pops_2010, county_data_merge, on=['County','State'])
county_pop_merge_2000 = pd.merge(state_pops_2000, county_data_merge, on=['County', 'State'])

In [21]:
#Getting all the data for each year in individual dataframes and then combining the new data frames

all_years = pd.DataFrame()
for year in range(2010, 2018):
    new_df = choose_data_by_year(county_pop_merge, year)
    all_years = concat_dfs_vertically(all_years, new_df)
    
for year in range(2000, 2010):
    new_df = choose_data_by_year(county_pop_merge_2000, year)
    all_years = concat_dfs_vertically(all_years, new_df)

In [22]:
#Calculating the % of total deaths
all_years['% of Total Deaths'] = np.int64(all_years['Deaths'])/all_years['Population']

In [23]:
#Creating a new column that changes the units to make them easier to deal with 
all_years['% of Total Deaths (x10^6)'] = all_years['% of Total Deaths'] * 1000000

In [24]:
all_years

,County,Population,County Code,Deaths,Month,Month Code,State,Year,% of Total Deaths,% of Total Deaths (x10^6)
137,Baldwin County,183111,01003,15,01,2010/01,Alabama,2010,0.000082,81.917525
138,Baldwin County,183111,01003,23,02,2010/02,Alabama,2010,0.000126,125.606872
139,Baldwin County,183111,01003,14,03,2010/03,Alabama,2010,0.000076,76.456357
140,Baldwin County,183111,01003,14,04,2010/04,Alabama,2010,0.000076,76.456357
141,Baldwin County,183111,01003,24,05,2010/05,Alabama,2010,0.000131,131.068041
142,Baldwin County,183111,01003,13,06,2010/06,Alabama,2010,0.000071,70.995189
143,Baldwin County,183111,01003,17,09,2010/09,Alabama,2010,0.000093,92.839862
144,Baldwin County,183111,01003,13,10,2010/10,Alabama,2010,0.000071,70.995189
145,Baldwin County,183111,01003,20,12,2010/12,Alabama,2010,0.000109,109.223367
183,Blount County,57373,01009,10,01,2010/01,Alabama,2010,0.000174,174.298015


In [25]:
all_years.isnull().values.any()

False